In [1]:
## Installing Spotify's API library in Python

# pip install spotipy

In [2]:
## Importing libraries

import sys
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from datetime import datetime

sys.path.append("/home/tabas/personal-dev/pyprojects")
import pipelines.utils.personal_env as penv
import pipelines.utils.common as common

In [3]:
# Importing Spotify Credentials

CLIENT_ID = penv.spotify_client_id
CLIENT_SECRET = penv.spotify_client_secret

In [4]:
# Stablishing Spotify Authentication

auth_manager = SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)
sp = spotipy.Spotify(auth_manager=auth_manager)

In [5]:
# Creating a list of all available markets on Spotify

markets = [
            "AD", "AE", "AG", "AL", #"AM", "AO", "AR", "AT", "AU", "AZ", "BA", "BB", "BD", 
            #"BE", "BF", "BG", "BH", "BI", "BJ", "BN", "BO", "BR", "BS", "BT", "BW", "BY", 
            #"BZ", "CA", "CD", "CG", "CH", "CI", "CL", "CM", "CO", "CR", "CV", "CW", "CY", 
            #"CZ", "DE", "DJ", "DK", "DM", "DO", "DZ", "EC", "EE", "EG", "ES", "ET", "FI", 
            #"FJ", "FM", "FR", "GA", "GB", "GD", "GE", "GH", "GM", "GN", "GQ", "GR", "GT", 
            #"GW", "GY", "HK", "HN", "HR", "HT", "HU", "ID", "IE", "IL", "IN", "IQ", "IS", 
            #"IT", "JM", "JO", "JP", "KE", "KG", "KH", "KI", "KM", "KN", "KR", "KW", "KZ", 
            #"LA", "LB", "LC", "LI", "LK", "LR", "LS", "LT", "LU", "LV", "LY", "MA", "MC", 
            #"MD", "ME", "MG", "MH", "MK", "ML", "MN", "MO", "MR", "MT", "MU", "MV", "MW", 
            #"MX", "MY", "MZ", "NA", "NE", "NG", "NI", "NL", "NO", "NP", "NR", "NZ", "OM", 
            #"PA", "PE", "PG", "PH", "PK", "PL", "PR", "PS", "PT", "PW", "PY", "QA", "RO", 
            #"RS", "RW", "SA", "SB", "SC", "SE", "SG", "SI", "SK", "SL", "SM", "SN", "SR", 
            #"ST", "SV", "SZ", "TD", "TG", "TH", "TJ", "TL", "TN", "TO", "TR", "TT", "TV", 
            #"TW", "TZ", "UA", "UG", "US", "UY", "UZ", "VC", "VE", "VN", "VU", "WS", "XK", 
            #"ZA", "ZM", "ZW"
           ]

### Getting latest releases

In [6]:
## Creating empty DataFrame to append API values after request

releases = pd.DataFrame()

In [7]:
## Creating loop to make GET Request
## The first request gets the list of new Albums released two weeks ago from each market defined above
## Then, it collects the ids of the Artists of each release and makes the second request, 
## which returns the Artist's data

for i in range(len(markets)):
    
    ## The Spotify only returns 50 values per request 
    # (the variables 'limit' and 'batchSize' helps Spotify not crash if the data exceeds )
    
    limit = 50
    offset = 0
    
    while offset < 1000:    # Spotify limit for Search Request is 1000
        
        ## Making GET request of the type search with the tag:'new', that returns the latest Albums

        newReleases = sp.search(q="tag:new", market=markets[i], type="album", limit=limit, offset=offset)
        newReleasesData = pd.DataFrame.from_dict(newReleases['albums']['items'])
        
        releases = pd.concat([releases, newReleasesData])
        releases['extractionTimestamp'] = datetime.today().strftime('%Y-%m-%d %X')
        
        # Incremental addition to offset to return the following pages of data
        
        offset=offset+limit
        
    print("Successfully got request from ", markets[i], "market")

Successfully got request from  AD market
Successfully got request from  AE market
Successfully got request from  AG market
Successfully got request from  AL market


In [8]:
# Returning table info

releases.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4000 entries, 0 to 49
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   album_type              4000 non-null   object
 1   total_tracks            4000 non-null   int64 
 2   is_playable             4000 non-null   bool  
 3   external_urls           4000 non-null   object
 4   href                    4000 non-null   object
 5   id                      4000 non-null   object
 6   images                  4000 non-null   object
 7   name                    4000 non-null   object
 8   release_date            4000 non-null   object
 9   release_date_precision  4000 non-null   object
 10  type                    4000 non-null   object
 11  uri                     4000 non-null   object
 12  artists                 4000 non-null   object
 13  extractionTimestamp     4000 non-null   object
dtypes: bool(1), int64(1), object(12)
memory usage: 441.4+ KB


In [9]:
## Treating some fields (renaming, extracting values, etc)

releases['spotify_url'] = releases['external_urls'].apply(lambda x: x['spotify'] if isinstance(x, dict) else None)

# Here, we are maintaining the id, uri and href in lists to facilitate in case we need to use them on the requests below

releases['artist_uri'] = releases['artists'].apply(lambda artists: [artist['uri'] for artist in artists])
releases['artist_href'] = releases['artists'].apply(lambda artists: [artist['href'] for artist in artists])
releases['artist_id'] = releases['artists'].apply(lambda artists: [artist['id'] for artist in artists])


In [10]:
# Dropping unnecessary and/or treated columns

releases = releases.drop(columns=['external_urls', 'artists'])

# Ordering columns

releases = releases[[
                    'id', 
                    'href', 
                    'uri', 
                    'spotify_url', 
                    'album_type', 
                    'total_tracks', 
                    'is_playable', 
                    'name', 
                    'release_date', 
                    'release_date_precision', 
                    'type', 
                    'artist_id', 
                    'artist_href', 
                    'artist_uri', 
                    'images', 
                    'extractionTimestamp']
            ]

In [11]:
# Removing duplicated release id 

releases = releases[releases.duplicated(subset='id') == False]

In [12]:
releases.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1349 entries, 0 to 23
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      1349 non-null   object
 1   href                    1349 non-null   object
 2   uri                     1349 non-null   object
 3   spotify_url             1349 non-null   object
 4   album_type              1349 non-null   object
 5   total_tracks            1349 non-null   int64 
 6   is_playable             1349 non-null   bool  
 7   name                    1349 non-null   object
 8   release_date            1349 non-null   object
 9   release_date_precision  1349 non-null   object
 10  type                    1349 non-null   object
 11  artist_id               1349 non-null   object
 12  artist_href             1349 non-null   object
 13  artist_uri              1349 non-null   object
 14  images                  1349 non-null   object
 15  extractionT

### Getting artists 

In [13]:
## Creating empty DataFrame to append API values after request

artists = pd.DataFrame()

In [14]:
## Making request to GET Artists' Data

# Here we're accessing the Artist ID to make the loop request below 
# We transforme the arrays into sets to remove duplicates, and then convert it back to lists, so it can be used
# on the API request
        
artistsList =  list(set(releases['artist_id'].explode()))

batchSize = 50 # Spotify limit for Artist Request is 50
        
for j in range(0, len(artistsList), batchSize):
    artistsBatch = artistsList[j:j + batchSize]
    artistsData = sp.artists(artistsBatch)
    artistsData = pd.DataFrame.from_dict(artistsData['artists'])  
    artists = pd.concat([artists, artistsData])
    artists['extractionTimestamp'] = datetime.today().strftime('%Y-%m-%d %X')
    
    print("Successfully got ", round(((j + batchSize)/len(artistsList)) * 100, 2), "% of Artists")

Successfully got  2.71 % of Artists
Successfully got  5.42 % of Artists
Successfully got  8.13 % of Artists
Successfully got  10.83 % of Artists
Successfully got  13.54 % of Artists
Successfully got  16.25 % of Artists
Successfully got  18.96 % of Artists
Successfully got  21.67 % of Artists
Successfully got  24.38 % of Artists
Successfully got  27.09 % of Artists
Successfully got  29.79 % of Artists
Successfully got  32.5 % of Artists
Successfully got  35.21 % of Artists
Successfully got  37.92 % of Artists
Successfully got  40.63 % of Artists
Successfully got  43.34 % of Artists
Successfully got  46.05 % of Artists
Successfully got  48.75 % of Artists
Successfully got  51.46 % of Artists
Successfully got  54.17 % of Artists
Successfully got  56.88 % of Artists
Successfully got  59.59 % of Artists
Successfully got  62.3 % of Artists
Successfully got  65.01 % of Artists
Successfully got  67.71 % of Artists
Successfully got  70.42 % of Artists
Successfully got  73.13 % of Artists
Succes

In [15]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1846 entries, 0 to 45
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   external_urls        1846 non-null   object
 1   followers            1846 non-null   object
 2   genres               1846 non-null   object
 3   href                 1846 non-null   object
 4   id                   1846 non-null   object
 5   images               1846 non-null   object
 6   name                 1846 non-null   object
 7   popularity           1846 non-null   int64 
 8   type                 1846 non-null   object
 9   uri                  1846 non-null   object
 10  extractionTimestamp  1846 non-null   object
dtypes: int64(1), object(10)
memory usage: 173.1+ KB


In [16]:
## Returning Artists DataFrame

artists.head()

,external_urls,followers,genres,href,id,images,name,popularity,type,uri,extractionTimestamp
0,{'spotify': 'https://open.spotify.com/artist/2...,"{'href': None, 'total': 169213}",[k-rap],https://api.spotify.com/v1/artists/2krUNMgFZYm...,2krUNMgFZYm5s4Nn0g91W9,[{'url': 'https://i.scdn.co/image/ab6761610000...,HAON,53,artist,spotify:artist:2krUNMgFZYm5s4Nn0g91W9,2025-02-07 11:01:29
1,{'spotify': 'https://open.spotify.com/artist/1...,"{'href': None, 'total': 17054}",[],https://api.spotify.com/v1/artists/1ZrJxKCLNjq...,1ZrJxKCLNjqk4B73SoR7PN,[{'url': 'https://i.scdn.co/image/ab6761610000...,Юпи,35,artist,spotify:artist:1ZrJxKCLNjqk4B73SoR7PN,2025-02-07 11:01:29
2,{'spotify': 'https://open.spotify.com/artist/6...,"{'href': None, 'total': 94283}","[epadunk, ragga]",https://api.spotify.com/v1/artists/6RjsbK9T7d1...,6RjsbK9T7d1UQD1PFEYYGt,[{'url': 'https://i.scdn.co/image/ab6761610000...,FRÖKEN SNUSK,56,artist,spotify:artist:6RjsbK9T7d1UQD1PFEYYGt,2025-02-07 11:01:29
3,{'spotify': 'https://open.spotify.com/artist/2...,"{'href': None, 'total': 4881}",[],https://api.spotify.com/v1/artists/2E41RSTUOR7...,2E41RSTUOR7CBYbfzXYI2t,[{'url': 'https://i.scdn.co/image/ab6761610000...,FEYI,26,artist,spotify:artist:2E41RSTUOR7CBYbfzXYI2t,2025-02-07 11:01:29
4,{'spotify': 'https://open.spotify.com/artist/0...,"{'href': None, 'total': 8158}",[],https://api.spotify.com/v1/artists/0A17cXaRbKp...,0A17cXaRbKpLKB4Eeqf5QE,[{'url': 'https://i.scdn.co/image/ab6761610000...,нестор,27,artist,spotify:artist:0A17cXaRbKpLKB4Eeqf5QE,2025-02-07 11:01:29


In [17]:
## Treating some fields (renaming, exploding the dicts, etc)

artists['spotify_url'] = artists['external_urls'].apply(lambda x: x['spotify'] if isinstance(x, dict) else None)
artists['followers'] = artists['followers'].apply(lambda x: x['total'] if isinstance(x, dict) else None)


In [18]:
# Dropping unnecessary and/or treated columns

artists = artists.drop(columns=['external_urls'])

# Ordering columns

artists = artists[[
                    'id', 
                    'href', 
                    'uri', 
                    'spotify_url', 
                    'type',
                    'name', 
                    'followers', 
                    'popularity', 
                    'genres', 
                    'images', 
                    'extractionTimestamp']
]

In [19]:
# Removing duplicated release id 

artists = artists[artists.duplicated(subset='id') == False]

In [20]:
artists.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1846 entries, 0 to 45
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   id                   1846 non-null   object
 1   href                 1846 non-null   object
 2   uri                  1846 non-null   object
 3   spotify_url          1846 non-null   object
 4   type                 1846 non-null   object
 5   name                 1846 non-null   object
 6   followers            1846 non-null   int64 
 7   popularity           1846 non-null   int64 
 8   genres               1846 non-null   object
 9   images               1846 non-null   object
 10  extractionTimestamp  1846 non-null   object
dtypes: int64(2), object(9)
memory usage: 173.1+ KB


### Getting albums 

In [21]:
## Let's get more data on this script
## Now that we have New Releases information, we can make a request to collect further information about albums and tracks
## Note that a new release can be an album or a single.
## So, to get more data, we're going to make a Request to return more informations about albums (like tracks, for example)
## And then, we are going to make a final request to return information about all tracks (single releases and tracks from the albums)

In [22]:
## Creating empty DataFrame to append API values after request

albums = pd.DataFrame()

In [23]:
# Again, using set and list to make the Album request below

albumsList = list(set(releases['id']))

# Redefining batchSize variable

batchSize = 20 # Spotify limit for Album Request is 20

for k in range(0, len(albumsList), batchSize):
    
    albumsData = sp.albums(albumsList[k:k + batchSize])
    albumsData = pd.DataFrame.from_dict(albumsData['albums'])
    albums = pd.concat([albums, albumsData])
    albums['extractionTimestamp'] = datetime.today().strftime('%Y-%m-%d %X')
    
    print("Successfully got ", round((k + batchSize) / len(albumsList) * 100, 2), "% of albums")

Successfully got  1.48 % of albums
Successfully got  2.97 % of albums
Successfully got  4.45 % of albums
Successfully got  5.93 % of albums
Successfully got  7.41 % of albums
Successfully got  8.9 % of albums
Successfully got  10.38 % of albums
Successfully got  11.86 % of albums
Successfully got  13.34 % of albums
Successfully got  14.83 % of albums
Successfully got  16.31 % of albums
Successfully got  17.79 % of albums
Successfully got  19.27 % of albums
Successfully got  20.76 % of albums
Successfully got  22.24 % of albums
Successfully got  23.72 % of albums
Successfully got  25.2 % of albums
Successfully got  26.69 % of albums
Successfully got  28.17 % of albums
Successfully got  29.65 % of albums
Successfully got  31.13 % of albums
Successfully got  32.62 % of albums
Successfully got  34.1 % of albums
Successfully got  35.58 % of albums
Successfully got  37.06 % of albums
Successfully got  38.55 % of albums
Successfully got  40.03 % of albums
Successfully got  41.51 % of albums
S

In [24]:
albums.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1349 entries, 0 to 8
Data columns (total 20 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   album_type              1349 non-null   object
 1   total_tracks            1349 non-null   int64 
 2   available_markets       1349 non-null   object
 3   external_urls           1349 non-null   object
 4   href                    1349 non-null   object
 5   id                      1349 non-null   object
 6   images                  1349 non-null   object
 7   name                    1349 non-null   object
 8   release_date            1349 non-null   object
 9   release_date_precision  1349 non-null   object
 10  type                    1349 non-null   object
 11  uri                     1349 non-null   object
 12  artists                 1349 non-null   object
 13  tracks                  1349 non-null   object
 14  copyrights              1349 non-null   object
 15  external_ids

In [25]:
albums.head()

,album_type,total_tracks,available_markets,external_urls,href,id,images,name,release_date,release_date_precision,type,uri,artists,tracks,copyrights,external_ids,genres,label,popularity,extractionTimestamp
0,single,1,"[AE, AM, AU, AZ, BD, BG, BH, BI, BN, BT, BW, C...",{'spotify': 'https://open.spotify.com/album/5g...,https://api.spotify.com/v1/albums/5gWtdeBPJnic...,5gWtdeBPJnicRkVORG8q5b,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Salute,2025-02-07,day,album,spotify:album:5gWtdeBPJnicRkVORG8q5b,[{'external_urls': {'spotify': 'https://open.s...,{'href': 'https://api.spotify.com/v1/albums/5g...,"[{'text': '© 2025 Balafon Productions AB, unde...",{'upc': '00602475895442'},[],Universal Music AB,0,2025-02-07 11:01:58
1,single,1,"[AM, AZ, BG, BY, CY, EE, FI, GE, GR, KZ, LT, L...",{'spotify': 'https://open.spotify.com/album/6k...,https://api.spotify.com/v1/albums/6ktCpNGwAMLr...,6ktCpNGwAMLr6KLLrQ8aDI,[{'url': 'https://i.scdn.co/image/ab67616d0000...,JLO,2025-02-07,day,album,spotify:album:6ktCpNGwAMLr6KLLrQ8aDI,[{'external_urls': {'spotify': 'https://open.s...,{'href': 'https://api.spotify.com/v1/albums/6k...,[{'text': '2025 Life is Pain under exclusive l...,{'upc': '3617669345334'},[],Life is Pain,0,2025-02-07 11:01:58
2,single,1,"[AD, AE, AG, AL, AM, AO, AR, AT, AU, AZ, BA, B...",{'spotify': 'https://open.spotify.com/album/1a...,https://api.spotify.com/v1/albums/1aDCSJhVHkQm...,1aDCSJhVHkQmbWbSQzRYiq,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Inamana,2025-02-06,day,album,spotify:album:1aDCSJhVHkQmbWbSQzRYiq,[{'external_urls': {'spotify': 'https://open.s...,{'href': 'https://api.spotify.com/v1/albums/1a...,[{'text': '© 2025 WASAFI LIMITED distributed e...,{'upc': '5021732628503'},[],Warner Music Africa,0,2025-02-07 11:01:58
3,single,1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",{'spotify': 'https://open.spotify.com/album/3X...,https://api.spotify.com/v1/albums/3XWtTmeGfsUM...,3XWtTmeGfsUMof0HvqvLEc,[{'url': 'https://i.scdn.co/image/ab67616d0000...,En Privado,2025-02-06,day,album,spotify:album:3XWtTmeGfsUMof0HvqvLEc,[{'external_urls': {'spotify': 'https://open.s...,{'href': 'https://api.spotify.com/v1/albums/3X...,"[{'text': '© 2025 Interscope Records', 'type':...",{'upc': '198704345081'},[],Interscope Records,0,2025-02-07 11:01:58
4,single,1,"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",{'spotify': 'https://open.spotify.com/album/6R...,https://api.spotify.com/v1/albums/6RhY6Snin8bK...,6RhY6Snin8bK6tfUHeadug,[{'url': 'https://i.scdn.co/image/ab67616d0000...,Gözlerime Bakmıyorsun,2025-02-07,day,album,spotify:album:6RhY6Snin8bK6tfUHeadug,[{'external_urls': {'spotify': 'https://open.s...,{'href': 'https://api.spotify.com/v1/albums/6R...,"[{'text': '2025 Fundyy', 'type': 'C'}, {'text'...",{'upc': '790092395459'},[],Fundyy,0,2025-02-07 11:01:58


In [26]:
## Treating some fields (renaming, extracting values, etc)

albums['spotify_url'] = albums['external_urls'].apply(lambda x: x['spotify'] if isinstance(x, dict) else None)

albums['artist_id'] = albums['artists'].apply(lambda artists: [artist['id'] for artist in artists])
albums['artist_href'] = albums['artists'].apply(lambda artists: [artist['href'] for artist in artists])
albums['artist_uri'] = albums['artists'].apply(lambda artists: [artist['uri'] for artist in artists])

## To extract the track id we'll have to:

# 1) Extract the key 'items' inside the dictionary

items = albums['tracks'].apply(lambda x: x['items'] if isinstance(x, dict) else [])

# 2) Extract the id inside the list

albums['track_id'] = items.apply(lambda x: [track['id'] for track in x] if isinstance(x, list) else [])
albums['track_href'] = items.apply(lambda x: [track['href'] for track in x] if isinstance(x, list) else [])
albums['track_uri'] = items.apply(lambda x: [track['uri'] for track in x] if isinstance(x, list) else [])

In [27]:
# Removing unnecessary and/or treated columns

albums = albums.drop(columns=['external_urls', 'artists', 'tracks', 'genres']) 

# Ordering columns

albums = albums[[
                'id', 
                'href',
                'uri',
                'spotify_url',
                'album_type',
                'total_tracks',  
                'name', 
                'available_markets',
                'release_date', 
                'release_date_precision', 
                'type', 
                'artist_id', 
                'artist_href', 
                'artist_uri', 
                'track_id',
                'track_href',
                'track_uri',
                'popularity',
                'label',
                'copyrights',
                'external_ids',                
                'images', 
                'extractionTimestamp']
            ]


In [28]:
# Removing duplicated release di 

albums = albums[albums.duplicated(subset='id') == False]

In [29]:
albums.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1349 entries, 0 to 8
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   id                      1349 non-null   object
 1   href                    1349 non-null   object
 2   uri                     1349 non-null   object
 3   spotify_url             1349 non-null   object
 4   album_type              1349 non-null   object
 5   total_tracks            1349 non-null   int64 
 6   name                    1349 non-null   object
 7   available_markets       1349 non-null   object
 8   release_date            1349 non-null   object
 9   release_date_precision  1349 non-null   object
 10  type                    1349 non-null   object
 11  artist_id               1349 non-null   object
 12  artist_href             1349 non-null   object
 13  artist_uri              1349 non-null   object
 14  track_id                1349 non-null   object
 15  track_href  

### Getting Tracks

In [30]:
## Now, we are going to get all tracks data
## Spotify provides general informations about Tracks, Audio Features and Audio Analysis

In [31]:
## Creating dataframe to append data

tracks = pd.DataFrame()

In [32]:
# Creating list of Track ids

tracksList = list(set(albums['track_id'].explode()))

# Redefining batchSize variable

batchSize = 50 # Spotify limit for Track Request is 50

for l in range(0, len(tracksList), batchSize):
    tracksData = sp.tracks(tracksList[l:l + batchSize])
    tracksData = pd.DataFrame.from_dict(tracksData['tracks'])
    
    tracks = pd.concat([tracks, tracksData])
    
    print("Successfully got ", round((l + batchSize) / len(tracksList) * 100, 2), "% of tracks")

Successfully got  1.08 % of tracks
Successfully got  2.17 % of tracks
Successfully got  3.25 % of tracks
Successfully got  4.34 % of tracks
Successfully got  5.42 % of tracks
Successfully got  6.51 % of tracks
Successfully got  7.59 % of tracks
Successfully got  8.68 % of tracks
Successfully got  9.76 % of tracks
Successfully got  10.85 % of tracks
Successfully got  11.93 % of tracks
Successfully got  13.02 % of tracks
Successfully got  14.1 % of tracks
Successfully got  15.18 % of tracks
Successfully got  16.27 % of tracks
Successfully got  17.35 % of tracks
Successfully got  18.44 % of tracks
Successfully got  19.52 % of tracks
Successfully got  20.61 % of tracks
Successfully got  21.69 % of tracks
Successfully got  22.78 % of tracks
Successfully got  23.86 % of tracks
Successfully got  24.95 % of tracks
Successfully got  26.03 % of tracks
Successfully got  27.11 % of tracks
Successfully got  28.2 % of tracks
Successfully got  29.28 % of tracks
Successfully got  30.37 % of tracks
Suc

In [33]:
tracks.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4610 entries, 0 to 9
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   album              4610 non-null   object
 1   artists            4610 non-null   object
 2   available_markets  4610 non-null   object
 3   disc_number        4610 non-null   int64 
 4   duration_ms        4610 non-null   int64 
 5   explicit           4610 non-null   bool  
 6   external_ids       4610 non-null   object
 7   external_urls      4610 non-null   object
 8   href               4610 non-null   object
 9   id                 4610 non-null   object
 10  is_local           4610 non-null   bool  
 11  name               4610 non-null   object
 12  popularity         4610 non-null   int64 
 13  preview_url        0 non-null      object
 14  track_number       4610 non-null   int64 
 15  type               4610 non-null   object
 16  uri                4610 non-null   object
dtypes: 

In [34]:
tracks.head()

,album,artists,available_markets,disc_number,duration_ms,explicit,external_ids,external_urls,href,id,is_local,name,popularity,preview_url,track_number,type,uri
0,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AM, AU, AZ, BD, BG, BH, BI, BN, BT, BW, C...",1,223022,True,{'isrc': 'ITF292400003'},{'spotify': 'https://open.spotify.com/track/6W...,https://api.spotify.com/v1/tracks/6W9k2Yb7xM5I...,6W9k2Yb7xM5IFRjjW7pXlb,False,Fiori d’arancio - Live in Siracusa,0,None,3,track,spotify:track:6W9k2Yb7xM5IFRjjW7pXlb
1,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AM, AU, AZ, BD, BG, BH, BI, BN, BT, BW, B...",1,202000,False,{'isrc': 'DET902590913'},{'spotify': 'https://open.spotify.com/track/1P...,https://api.spotify.com/v1/tracks/1PLxIpvlVZUl...,1PLxIpvlVZUlPmF5azvHe4,False,"Symphony No. 6 in F Major, K. 43: IV. Allegro",0,None,13,track,spotify:track:1PLxIpvlVZUlPmF5azvHe4
2,"{'album_type': 'single', 'artists': [{'externa...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AM, AU, AZ, BD, BG, BH, BI, BN, BT, BW, B...",1,171973,False,{'isrc': 'NLE872513918'},{'spotify': 'https://open.spotify.com/track/6U...,https://api.spotify.com/v1/tracks/6UnhUeIjMQhu...,6UnhUeIjMQhuLNLVQzj5zc,False,De Prins - Bremspoersengat - Jeugdprins Jochem...,0,None,18,track,spotify:track:6UnhUeIjMQhuLNLVQzj5zc
3,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AM, AU, AZ, BD, BG, BH, BI, BN, BT, BW, C...",1,167062,True,{'isrc': 'FI8RM2500013'},{'spotify': 'https://open.spotify.com/track/0F...,https://api.spotify.com/v1/tracks/0FzpBONOXM55...,0FzpBONOXM55bzOQ1ldmSL,False,Päätepysäkki,0,None,13,track,spotify:track:0FzpBONOXM55bzOQ1ldmSL
4,"{'album_type': 'album', 'artists': [{'external...",[{'external_urls': {'spotify': 'https://open.s...,"[AE, AM, AU, AZ, BD, BG, BH, BI, BN, BT, BW, C...",1,189853,False,{'isrc': 'CAW111200084'},{'spotify': 'https://open.spotify.com/track/3n...,https://api.spotify.com/v1/tracks/3noE07Mk4Utj...,3noE07Mk4Utjy1k83VTFfE,False,What Am I Doing Here - 2012 Remaster,0,None,2,track,spotify:track:3noE07Mk4Utjy1k83VTFfE


In [35]:
tracks['spotify_url'] = tracks['external_urls'].apply(lambda x: x['spotify'] if isinstance(x, dict) else None)


tracks['artists_id'] = tracks['artists'].apply(lambda artists: [artist['id'] for artist in artists])
tracks['artists_href'] = tracks['artists'].apply(lambda artists: [artist['href'] for artist in artists])
tracks['artists_uri'] = tracks['artists'].apply(lambda artists: [artist['uri'] for artist in artists])

tracks['album_id'] = tracks['album'].apply(lambda x: x['id'] if isinstance(x, dict) else [])
tracks['album_href'] = tracks['album'].apply(lambda x: x['href'] if isinstance(x, dict) else [])
tracks['album_uri'] = tracks['album'].apply(lambda x: x['uri'] if isinstance(x, dict) else [])

In [36]:
tracks = tracks.drop(columns = ['album', 'artists', 'external_urls'])

tracks = tracks[[
                'id',
                'href',
                'uri',
                'spotify_url',
                'artists_id',
                'artists_href',
                'artists_uri',
                'album_id',
                'album_href',
                'album_uri',
                'available_markets',
                'disc_number',
                'duration_ms',
                'explicit',
                'external_ids',]           
    ]

### Unloading Data

In [37]:
## Importing Credentials from Google Cloud

from google.cloud import storage
from google.oauth2 import service_account

CREDENTIALS = service_account.Credentials.from_service_account_file(penv.bq_path)
STORAGE = storage.Client(credentials=CREDENTIALS)

In [40]:
## Defining variables to execute unload_data()

bucket = STORAGE.get_bucket(penv.bucket_path)
bucket_folder = penv.bucket_folder
file_formats = ['parquet']
df_dict = {
        'releases': releases,
        'artists': artists,
        'albums': albums,
        'tracks': tracks    
}

In [41]:
## Writing Dataframe to Bucket folder with desired file format 

common.unload_data(file_formats, df_dict, bucket, bucket_folder)


Begin at:  2025-02-07 11:05:01
Sucessfully written  releases  in  parquet
End at:  2025-02-07 11:05:02

Begin at:  2025-02-07 11:05:02
Sucessfully written  artists  in  parquet
End at:  2025-02-07 11:05:03

Begin at:  2025-02-07 11:05:03
Sucessfully written  albums  in  parquet
End at:  2025-02-07 11:05:05

Begin at:  2025-02-07 11:05:05
Sucessfully written  tracks  in  parquet
End at:  2025-02-07 11:05:06


In [42]:
## This section is focused on understanding the data to find if further treatment is necessary
## Throuhgout the notebook, I've made some treatments base on what I've seen executiong the commands here in the notebook
## , but now I'm going to bring each dataframe (releases, artists, albums, tracks) to BigQuery and do some querying there

In [43]:
""" Releases (before removing duplicated values)

The releases data has the same structure as the album's data. Although the later has additional info, such as popularity
  , available markets, label, copyrights, and others.

Run made 06/02/2025 10:05

- album_type: 'single', 'album', 'compilation';
- release_date: minimum value is 2 weeks before today, maximum value is one day ahead;
- release_date_precision: take only value 'day';
- type: take only value 'album';


- Querying the distinct data, we get 1428 unique releas1es and 1507 unique artists;
SELECT 
  COUNT(DISTINCT id) AS qtd_release
  , COUNT(DISTINCT element) AS qtd_artists
FROM df_releases` 
  , UNNEST(artist_id.list) AS art_id
LIMIT 1000

- Checking if duplicated ids have indeed all the same values on all the columns id = '0hYiEucmpqclKoXxkBiIPS' as example) 
SELECT 
  *
FROM `tabas-dw-stg.stg_juliana.test_releases` 
  , UNNEST(artist_id.list) AS art_id
WHERE id = '0hYiEucmpqclKoXxkBiIPS' 

"""

" Releases (before removing duplicated values)\n\nThe releases data has the same structure as the album's data. Although the later has additional info, such as popularity\n  , available markets, label, copyrights, and others.\n\nRun made 06/02/2025 10:05\n\n- album_type: 'single', 'album', 'compilation';\n- release_date: minimum value is 2 weeks before today, maximum value is one day ahead;\n- release_date_precision: take only value 'day';\n- type: take only value 'album';\n\n\n- Querying the distinct data, we get 1428 unique releas1es and 1507 unique artists;\nSELECT \n  COUNT(DISTINCT id) AS qtd_release\n  , COUNT(DISTINCT element) AS qtd_artists\nFROM df_releases` \n  , UNNEST(artist_id.list) AS art_id\nLIMIT 1000\n\n- Checking if duplicated ids have indeed all the same values on all the columns id = '0hYiEucmpqclKoXxkBiIPS' as example) \nSELECT \n  *\nFROM `tabas-dw-stg.stg_juliana.test_releases` \n  , UNNEST(artist_id.list) AS art_id\nWHERE id = '0hYiEucmpqclKoXxkBiIPS' \n\n"

In [44]:
""" Artists 

Run made 06/02/2025 10:07

- genres: list of values - initially, I treated this field extracting the values from the list and maintaining 
        them solely on comma separated values, but on BigQuery it's possible to unnest values easily, so I 
        changed it back to original format;
- type: take only value 'artist';
- followers: total followers the artist has on Spotify profile - initially, the amount was divergent from what it was
        being shown on Spotify's page. I used json_normalize() to treat this field, but on investigation I've discovered 
        that it was atributing the wrong values. Using lambda function, it worked;
- popularity: goes from 0 to 100 - the artist's popularity is calculated from the popularity of all the artist's tracks.

- After running artists dataframe, we realize that it returned 1507 artists (as expected from above).

"""

" Artists \n\nRun made 06/02/2025 10:07\n\n- genres: list of values - initially, I treated this field extracting the values from the list and maintaining \n        them solely on comma separated values, but on BigQuery it's possible to unnest values easily, so I \n        changed it back to original format;\n- type: take only value 'artist';\n- followers: total followers the artist has on Spotify profile - initially, the amount was divergent from what it was\n        being shown on Spotify's page. I used json_normalize() to treat this field, but on investigation I've discovered \n        that it was atributing the wrong values. Using lambda function, it worked;\n- popularity: goes from 0 to 100 - the artist's popularity is calculated from the popularity of all the artist's tracks.\n\n- After running artists dataframe, we realize that it returned 1507 artists (as expected from above).\n\n"

In [45]:
""" Albums 

Run made 06/02/2025 11:07

- track_ids: Due to same issue of json_normalize() mentioned above, this field was returning a lot of duplicated
            values. Once fixed with lambda functions, it is consistent.

- After running artists dataframe, we realize that it returned 1428 albums (as expected from above).

"""

' Albums \n\nRun made 06/02/2025 11:07\n\n- track_ids: Due to same issue of json_normalize() mentioned above, this field was returning a lot of duplicated\n            values. Once fixed with lambda functions, it is consistent.\n\n- After running artists dataframe, we realize that it returned 1428 albums (as expected from above).\n\n'